In [1]:
import pandas as pd
import numpy as np
import pickle
from collections import defaultdict
from tqdm import tqdm
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import math

In [2]:
with open('../inference/27-02-2024_2352_antikey_output.pkl','rb') as file:
    impact_output = pickle.load(file)
with open('../Data/index_label_mapping.pkl','rb') as file:
    index_label_mapping = pickle.load(file)

In [3]:
len(impact_output)

669785

In [4]:
unique_values = {x.replace("Job ",""):list(index_label_mapping[x].values()) for x in index_label_mapping}
unique_values

{'Role': ['DEVELOPMENT',
  'GOVERNANCE RISK COMPLIANCE',
  'INFORMATION SECURITY',
  'IT GENERAL',
  'NETWORKING',
  'NON-ICP',
  'SYSTEMS'],
 'Function': ['ENGINEERING',
  'IT',
  'NON-ICP',
  'PROCUREMENT',
  'RISK/LEGAL/COMPLIANCE'],
 'Level': ['C-LEVEL',
  'CONTRIBUTOR',
  'DIRECTOR',
  'EXECUTIVE',
  'MANAGER',
  'UNKNOWN']}

In [5]:
impact_output[0]

{'Sequence': 'IT DIRECTOR',
 'Role': [{'Anti-Prediction': 'DEVELOPMENT',
   'Distinct_Tokens': ['IT', 'DIRECTOR'],
   'Token_Importance': {'IT': 0.052871253341436386,
    'DIRECTOR': 0.9471287727355957},
   'Token_Rank': {'IT': 2, 'DIRECTOR': 1},
   'Token_Marginal_Score_Positive': {'IT': 0.07268381118774414,
    'DIRECTOR': 1.3020483255386353},
   'Token_Marginal_Score_Raw': {'IT': 0.07268381118774414,
    'DIRECTOR': 1.3020483255386353}},
  {'Anti-Prediction': 'GOVERNANCE RISK COMPLIANCE',
   'Distinct_Tokens': ['IT', 'DIRECTOR'],
   'Token_Importance': {'IT': 0.46622586250305176,
    'DIRECTOR': 0.5337741374969482},
   'Token_Rank': {'IT': 2, 'DIRECTOR': 1},
   'Token_Marginal_Score_Positive': {'IT': 1.185859203338623,
    'DIRECTOR': 1.3576703071594238},
   'Token_Marginal_Score_Raw': {'IT': 1.185859203338623,
    'DIRECTOR': 1.3576703071594238}},
  {'Anti-Prediction': 'INFORMATION SECURITY',
   'Distinct_Tokens': ['IT', 'DIRECTOR'],
   'Token_Importance': {'IT': 0.0, 'DIRECTOR': 1

In [6]:
# Initialize compilation dictionary - we'll have entries for role, function, and level, which will lead to
# entries for each potential output category, which will lead to entries for every individual word with:
# 1. Average token importance
# 2. Average number of unique tokens in sequence
# 3. Average marginal score (positive) - keyword
# 4. Average marginal score (raw)
# 5. Average marginal score (negative) - anti-keyword
# 6. Average token score rank
# For 1-6 we'll first need to just record every entry in the structure, then we can run through and take the average
keyword_dict_running = {**unique_values}
for x in keyword_dict_running:
    keyword_dict_running[x] = {}
    for y in unique_values[x]:
        keyword_dict_running[x][y] = {}

In [7]:
# Run through the data
# tqdm(enumerate(data_loader,0),total=len(data_loader))
for _,sequence in tqdm(enumerate(impact_output,0),total=len(impact_output)):
    for key in keyword_dict_running:
        this_sequence_info = sequence[key]
        for anti_prediction in this_sequence_info:
            this_anti_prediction = anti_prediction['Anti-Prediction']
            this_tokens = anti_prediction['Distinct_Tokens']
            this_unique_tokens_count = len(this_tokens)
            total_score_difference = np.array(list(anti_prediction['Token_Marginal_Score_Positive'].values())).sum()
            average_score_difference = total_score_difference/this_unique_tokens_count
            for token in this_tokens:
                # If not already present in the keyword_dict_running, add it
                if token not in keyword_dict_running[key][this_anti_prediction]:
                    keyword_dict_running[key][this_anti_prediction][token] = defaultdict(list)
                # Append to lists the abovementioned metrics
                token_importance = anti_prediction['Token_Importance'][token]
                marginal_score_positive = anti_prediction['Token_Marginal_Score_Positive'][token]
                raw_score = anti_prediction['Token_Marginal_Score_Raw'][token]
                token_rank = anti_prediction['Token_Rank'][token]
                priority_index = marginal_score_positive/average_score_difference
                if np.isnan(priority_index):
                    priority_index = 0
                excess_ri = max(0,priority_index - 1)
                if math.isnan(token_importance):
                    token_importance = 0
                keyword_dict_running[key][this_anti_prediction][token]['Token_Importance'].append(token_importance)
                keyword_dict_running[key][this_anti_prediction][token]['Unique_Tokens_Count'].append(this_unique_tokens_count)
                keyword_dict_running[key][this_anti_prediction][token]['Marginal_Score_Positive'].append(marginal_score_positive)
                keyword_dict_running[key][this_anti_prediction][token]['Marginal_Score_Raw'].append(raw_score)
                keyword_dict_running[key][this_anti_prediction][token]['Marginal_Score_Negative'].append(min(raw_score,0))
                keyword_dict_running[key][this_anti_prediction][token]['Token_Rank'].append(token_rank)
                keyword_dict_running[key][this_anti_prediction][token]['Token_Occurrences'].append(1)
                keyword_dict_running[key][this_anti_prediction][token]['Priority_Index_>_1'].append((priority_index > 1)*1)
                keyword_dict_running[key][this_anti_prediction][token]['Priority_Index'].append(priority_index)

  0%|          | 0/669785 [00:00<?, ?it/s]C:\Users\csarc\AppData\Local\Temp\ipykernel_32472\2607293029.py:21: RuntimeWarning: invalid value encountered in scalar divide
  priority_index = marginal_score_positive/average_score_difference
100%|██████████| 669785/669785 [07:04<00:00, 1578.78it/s]


In [8]:
keyword_dict_average = {**unique_values}
for x in keyword_dict_average:
    keyword_dict_average[x] = {}
    for y in unique_values[x]:
        keyword_dict_average[x][y] = {}

In [9]:
# Now we run through the above dictionary and calculate the averages for each token and underlying list
for key in keyword_dict_running:
    for prediction in keyword_dict_running[key]:
        keyword_dict_average[key][prediction] = defaultdict(list)
        for _,token in tqdm(enumerate(keyword_dict_running[key][prediction])):
            priority_index = np.array(keyword_dict_running[key][prediction][token]['Priority_Index'])
            token_occurrences = np.array(keyword_dict_running[key][prediction][token]['Token_Occurrences'])
            pi_g_1 = np.array(keyword_dict_running[key][prediction][token]['Priority_Index_>_1'])
            keyword_dict_average[key][prediction]['Token'].append(token)
            keyword_dict_average[key][prediction]['Total_Token_Occurrences'].append(np.array(keyword_dict_running[key][prediction][token]['Token_Occurrences']).sum())
            keyword_dict_average[key][prediction]['Prob_Priority_Index_>_1'].append(pi_g_1.sum()/token_occurrences.sum())
            keyword_dict_average[key][prediction]['StDev_Priority_Index'].append(np.std(priority_index))
            keyword_dict_average[key][prediction]['Avg_Priority_Index'].append(np.mean(priority_index))

23166it [00:01, 12749.92it/s]
24527it [00:01, 13722.65it/s]
20237it [00:01, 14285.75it/s]
23672it [00:01, 12700.35it/s]
21156it [00:01, 14571.54it/s]
19026it [00:01, 12260.59it/s]
24313it [00:01, 12614.64it/s]
23240it [00:01, 13170.78it/s]
12822it [00:00, 19374.58it/s]
19476it [00:01, 12155.63it/s]
24735it [00:01, 13293.37it/s]
24763it [00:01, 12975.01it/s]
23020it [00:01, 13623.70it/s]
18873it [00:01, 13607.03it/s]
21653it [00:01, 13509.03it/s]
21800it [00:01, 13906.28it/s]
21198it [00:01, 14014.30it/s]
24606it [00:01, 12726.84it/s]


In [10]:
# Now we create a table for each output and possible value for that output. Let's start with a new dictionary to house it
keyword_table_dict = {}
for x in unique_values:
    keyword_table_dict[x] = {}

In [11]:
for key in keyword_dict_average:
    for output in keyword_dict_average[key]:
        keyword_table_dict[key][output] = pd.DataFrame.from_dict(keyword_dict_average[key][output])

In [12]:
# Write tables to excel - filter out values with less than 100 total token occurrences
writer_keyword = pd.ExcelWriter('../inference/impact_output_antikeyword_ALTERNATE.xlsx',engine = 'xlsxwriter')
for key in keyword_table_dict:
    for output in keyword_table_dict[key]:
        this_df = keyword_table_dict[key][output] 
        this_df[(this_df.Total_Token_Occurrences >= 100) & (this_df.Avg_Priority_Index > 1)].sort_values(by = 'Avg_Priority_Index',ascending=False).to_excel(
            writer_keyword,sheet_name='{}_{}'.format(key.replace('/',''),output.replace('/','')),index = False)
writer_keyword.close()